In [3]:
from bs4 import BeautifulSoup as bs
import requests as req
from pprint import pprint
from selenium import webdriver
from time import sleep

import os, json
import re
import regex


# 抓書名網址
url = 'https://www.gutenberg.org/browse/languages/zh'
res = req.get(url)
res.encoding = 'utf-8'
soup = bs(res.text, 'lxml')

# pprint(soup.get_text)


# 取書名
book_name_list=[]
bn_list = soup.select("li.pgdbetext a")

#去除英文書名
for bn in bn_list[:300]:
    bn_desc = ''.join(regex.findall( r'[\p{Han}\p{P}]+', bn.get_text()))
    book_name_list.append(bn_desc)
# pprint(book_name_list)

# 取文字內容
book_text_list =[]  
book_text_link = soup.select("li.pgdbetext a[href]")

#得到內容網址
for bt_link in book_text_link[:300]:
    book_content = ""
    bt_link = re.search(r'/ebooks/(\d+)', bt_link["href"])
    url2 = f'https://www.gutenberg.org/cache/epub/{bt_link[1]}/pg{bt_link[1]}-images.html'

    #抓內容網誌
    res2 = req.get(url2)
    res2.encoding = 'utf-8'
    soup2 = bs(res2.text, 'lxml')
    link2_p_list = soup2.select('body p')

    #篩選內容，緊抓取中文內容及標點符號
    for link2_p in link2_p_list:
        desc = ''.join(regex.findall( r'[\p{Han}\p{P}]+', link2_p.get_text()))
        book_content += desc if len(desc) > 10 else ""
    book_text_list.append(book_content)

#將 書名 及 文字內容 去掉空格+重覆 後整理
result_list = []
seen_names = set()
for bn, bt in zip(book_name_list, book_text_list):
    clean_name = bn.strip()
    if not clean_name or clean_name in seen_names:
        continue
    result_list.append({
    "name" : clean_name,
    "text" : bt
    })
    seen_names.add(clean_name)
# pprint(result_list)

#新增資料夾
folderPath = 'project_gutenberg'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
    
#整理書名
for book in result_list:
    filename = book['name'].strip()
    for ch in r'\/:*?"<>|':
        filename = filename.replace(ch, "_")

#下載TXT檔
    filepath = os.path.join(folderPath, f"{filename}.txt")

    with open(filepath, "w", encoding="utf-8") as f:
        f.write(book['text'])

    print(f"已儲存：{filepath}")


已儲存：project_gutenberg\豆棚閒話.txt
已儲存：project_gutenberg\戲中戲.txt
已儲存：project_gutenberg\比目魚.txt
已儲存：project_gutenberg\三字經.txt
已儲存：project_gutenberg\山水情.txt
已儲存：project_gutenberg\山海經.txt
已儲存：project_gutenberg\施公案.txt
已儲存：project_gutenberg\易經.txt
已儲存：project_gutenberg\木蘭奇女傳.txt
已儲存：project_gutenberg\海公案.txt
已儲存：project_gutenberg\燕丹子.txt
已儲存：project_gutenberg\狄公案.txt
已儲存：project_gutenberg\百家姓.txt
已儲存：project_gutenberg\禮記.txt
已儲存：project_gutenberg\綠牡丹.txt
已儲存：project_gutenberg\詩經.txt
已儲存：project_gutenberg\麟兒報.txt
已儲存：project_gutenberg\天豹圖.txt
已儲存：project_gutenberg\梁公九諫.txt
已儲存：project_gutenberg\長恨歌.txt
已儲存：project_gutenberg\李娃傳.txt
已儲存：project_gutenberg\玉樓春.txt
已儲存：project_gutenberg\漢書.txt
已儲存：project_gutenberg\引鳳蕭.txt
已儲存：project_gutenberg\今古奇觀.txt
已儲存：project_gutenberg\後西游記.txt
已儲存：project_gutenberg\飛跎全傳.txt
已儲存：project_gutenberg\佛說四十二章經.txt
已儲存：project_gutenberg\紅樓夢.txt
已儲存：project_gutenberg\洛神賦.txt
已儲存：project_gutenberg\晁氏儒言一卷.txt
已儲存：project_gutenberg\水滸後傳.txt
已儲存：project_gutenberg\幼學瓊林.tx